In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.utils.data import get_pkg_data_filename
from pathlib import Path

from superwasp import lightcurve

In [ ]:
plt.style.use('guide.mplstyle')

In [ ]:
object_name = 'CRTS J144508.6+050514'
filename = Path('/data/SuperWASP/1SWASP J144508.70+050514.4.fits')

In [ ]:
lc = lightcurve(filename)

In [ ]:
#lc.normalize()
fig, ax1 = lc.plot(show_mean=True)
ax1.set_title(object_name + ', period = ' + str(lc.period.to(u.hour)))

In [ ]:
lc.write(filename.with_suffix('.csv'), format='ascii.csv', overwrite=True)

In [ ]:
fig.savefig(filename.with_suffix('.png'), format='png')